## Imports

In [1]:
import karman
import os
import pickle
import torch

## Loading Dataset

In this case, we show how to load a dataset that contains NRLMSISE-00 and JB-08 inputs (latitude, longitude, altitude, corresponding thermospheric density, for GRACE, GOCE, SWARM-A, SWARM-B, CHAMP satellites, F10.7, Y10.7, M10.7, S10.7 Proxies, Ap indices, temperature changes due to Dst, etc.).

In [15]:
base_path='/'+os.getcwd().strip('docs/source/tutorials')
#we first load the scalers used in training
with open(os.path.join(base_path,"tests/data/thermo_scaler.pickle"), 'rb') as handle:
    thermo_scaler = pickle.load(handle)

with open(os.path.join(base_path,"tests/data/cyclical_feature_scaler.pickle"), 'rb') as handle:
    cyclical_feature_scaler = pickle.load(handle)

with open(os.path.join(base_path,"tests/data/thermospheric_density_scaler.pickle"), 'rb') as handle:
    thermospheric_density_scaler = pickle.load(handle)

In [17]:
#we store the features to exclude (not used in training):
#we exclude Ap because we trained an ML model using only JB-08 inputs
features_to_exclude_thermo=['all__dates_datetime__',
                            'tudelft_thermo__ground_truth_thermospheric_density__[kg/m**3]',
                            'tudelft_thermo__satellite__',
                            'celestrack__ap_h_0__',
                            'celestrack__ap_h_1__',
                            'celestrack__ap_h_2__',
                            'celestrack__ap_h_3__',
                            'celestrack__ap_h_4__',
                            'celestrack__ap_h_5__',
                            'celestrack__ap_h_6__',
                            'celestrack__ap_average__',
                            'JB08__thermospheric_density__[kg/m**3]',
                            'NRLMSISE00__thermospheric_density__[kg/m**3]']
#we now load the dataset:
dataset=karman.ThermosphericDensityDataset(
    directory=os.path.join(base_path,'tests/data/'),
    features_to_exclude_thermo=features_to_exclude_thermo,
    create_cyclical_features=True,
    max_altitude=600000,
    thermo_scaler=thermo_scaler,
    cyclical_feature_scaler=cyclical_feature_scaler,
    thermospheric_density_scaler=thermospheric_density_scaler
)

Creating thermospheric density dataset
Creating cyclical features
['all__dates_datetime__', 'tudelft_thermo__ground_truth_thermospheric_density__[kg/m**3]', 'tudelft_thermo__satellite__', 'celestrack__ap_h_0__', 'celestrack__ap_h_1__', 'celestrack__ap_h_2__', 'celestrack__ap_h_3__', 'celestrack__ap_h_4__', 'celestrack__ap_h_5__', 'celestrack__ap_h_6__', 'celestrack__ap_average__', 'JB08__thermospheric_density__[kg/m**3]', 'NRLMSISE00__thermospheric_density__[kg/m**3]']
['all__day_of_year__[d]', 'all__seconds_in_day__[s]', 'all__sun_right_ascension__[rad]', 'all__sun_declination__[rad]', 'all__sidereal_time__[rad]', 'tudelft_thermo__longitude__[deg]', 'tudelft_thermo__local_solar_time__[h]']
Used features: Index(['all__year__[y]', 'tudelft_thermo__altitude__[m]',
       'tudelft_thermo__latitude__[deg]',
       'space_environment_technologies__f107_average__',
       'space_environment_technologies__f107_obs__',
       'space_environment_technologies__s107_obs__',
       'space_environm

## Loading Karman-ML Thermospheric Density Model

In [21]:
#we now load the Karman ML trained model:
model_path=os.path.join(base_path,"tests/data/kml_model")
print('Loading Model')

model = karman.nn.SimpleNN().to(dtype=torch.float64)

state_dict = torch.load(os.path.join(model_path),map_location=torch.device('cpu'))['state_dict']
#Sanitize state_dict key names
for key in list(state_dict.keys()):
    if key.startswith('module'):
    # Model was saved as dataparallel model
        # Remove 'module.' from start of key
        state_dict[key[7:]] = state_dict.pop(key)
    else:
        continue
model.load_state_dict(state_dict)

Loading Model


<All keys matched successfully>